In [ ]:
###-----------------
### Import Libraries
###-----------------
import sys


sys.path.append('../')
 

from sklearn import datasets
from sklearn.model_selection import train_test_split
###-----------------
### Import Libraries
###-----------------
import os
import time
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
from utils.helper import fn_plot_tf_hist, fn_plot_confusion_matrix
import gc
import torch.nn as nn
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

import tensorflow as tf
from utils.helper import fn_plot_torch_hist, fn_plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Global variables
inpDir = '../input' # Input Stored here
outDir = '../ouput' # output Here
modelDir = '../models'# to save Models
subDir = 'fifa_2019' # sub dir by dataset
RANDOM_STATE = 24
np.random.RandomState(seed = RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
#rng = np.random.default_rng(seed = RANDOM_STATE)
#N_SAMPLE = 1000
TEST_SIZE = 0.2
ALPHA = 0.001 # learning rate
NOISE = 0.2 # Error
EPOCHS = 101
BATCH_SIZE = 256
LR_FACTOR=0.1
LR_PATIENCE=10

In [ ]:
#set the plotting parameters
params = {
    'legend.fontsize': 'medium',
    'figure.figsize':(15,4),
    'axes.labelsize':'medium',
    'axes.titlesize':'medium',
    'xtick.labelsize': 'medium',
    'ytick.labelsize':'medium',
    #'text.usetex':True,
}
plt.rcParams.update(params)
CMAP = plt.cm.coolwarm
plt.style.use('seaborn-v0_8-darkgrid')


In [ ]:
# data_df = pd.read_csv(os.path.join(inpDir,'fifa_2019.csv'))

In [ ]:
train_filename = os.path.join(inpDir, 'fashion_mnist', 'fashion-mnist_train.csv')
test_filename = os.path.join(inpDir, 'fashion_mnist', 'fashion-mnist_test.csv')

train_df = pd.read_csv(train_filename, header = 0)
test_df = pd.read_csv(test_filename, header = 0)

In [ ]:
train_df

In [ ]:
test_df.head()

In [ ]:
train_df.isnull().sum().sum(), test_df.isnull().sum().sum()

In [ ]:
#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

class_names = {0: 'T-shirt/top',1:'Trouser',2:'Pullover',3:'Dress',4:'Coat',
               5:'Sandal', 6: 'Shirt',7: 'Sneaker', 8:'Bag', 9: 'Ankle boot'}

In [ ]:
# sample DATA plot: Display a grid of images from the dataset along with their labels

n_rows =  8 # Number of rows to display in the grid
n_cols = 8 # Number of columns

# randomly sample 'n_rows' * n_cols images from the dataset
plot_df = train_df.sample(n = n_rows * n_cols)

#create a new figure for the grid of images with a specified size
fig = plt.figure(figsize  =(15,12))

#Adjust Margins
fig.subplots_adjust(left = 0, right = 1, bottom=0, top = 1, hspace =0.05, wspace = 0.05)
i= 0 
for idx, row in plot_df.iterrows():
    i += 1
    image = row.values[1:].reshape(28,28)

    ax = fig.add_subplot(n_rows, n_cols, i, xticks=[], yticks=[])
    #Display the image on the subplot using a binary colormap
    ax.imshow(image, cmap = plt.cm.binary, interpolation = 'nearest')

    ax.text(2,4, str(row.iloc[0]), color = 'b', fontsize=16)
    #
    ax.text(2,25, class_names[row.iloc[0]], color='r', fontsize  = 16)
#Display the entire grid of images
plt.show()

In [ ]:
plot_df.shape

In [ ]:
# X = data_df.drop('Position',axis=1)
# y = data_df['Position']

In [ ]:
# data_df.dropna(subset= ('Position'), inplace = True)


In [ ]:
for col in train_df.columns:
    unq = train_df[col].unique()
    print(f'{col}, #:{len(unq)}, Values:{unq}')

In [ ]:
train_df.columns

In [ ]:
train_df.info()


In [ ]:
# num_cols = data_df.select_dtypes(exclude='object')

In [ ]:
# num_cols.info()

In [ ]:
X_data = train_df.drop('label',axis=1).to_numpy()
y_data = train_df['label'].to_numpy()


## Split Data in test train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=TEST_SIZE,stratify = y_data, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_valid = test_df.drop('label', axis=1).to_numpy()
y_valid = test_df['label'].to_numpy()

In [ ]:
type(X_train), type(X_test), type(y_train), type(y_test)

In [ ]:
X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape

In [ ]:
# Normalize data beetween [0-1]
X_train = X_train / 255.0
X_test = X_test / 255.0
X_valid = X_valid / 255.0

In [ ]:
# # to delete the data which not useful any more
# del train_df, test_df, X_train , X_test, X_valid, X_data, y_data
# gc.collect()
# gc.collect()

In [ ]:
# mm = MinMaxScaler()
# X_train = mm.fit_transform(X_train)
# X_test = mm.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
'''Define custom dataset'''
class MNISTDataset(Dataset):
    def __init__(self, X, y):
        super(MNISTDataset, self).__init__()
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.long)
    def __len__(self): # length of the data = no. of rows 
        return(len(self.X))

    def __getitem__(self, idx): # give me index of X , y
        return self.X[idx], self.y[idx]
    




In [ ]:
train_dataset = MNISTDataset(X_train, y_train)

In [ ]:
train_loader = DataLoader(
    dataset = train_dataset, 
    batch_size = BATCH_SIZE,
    shuffle = True
)

In [ ]:
X_batch , y_batch = next(iter(train_loader))
X_batch.shape, y_batch.shape


In [ ]:
test_dataset = MNISTDataset(X_test, y_test)

In [ ]:
test_loader = DataLoader(
    dataset = test_dataset, 
    batch_size = BATCH_SIZE,
    shuffle = True
)
X_batch, y_batch = next(iter(test_loader))
X_batch.shape, y_batch.shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
input_dim = X_train.shape[1]

class Model(nn.Module):
    # Dropout
    #batchNorm
    
    def __init__(self, input_dim):
        super(Model, self).__init__()
        dor = [0.2,0.3,0.4,0.5,0.6]
        #set1   
        self.layer1 = nn.Linear(input_dim, 392)
        self.actv1 = nn.ReLU()
        self.do1 = nn.Dropout(dor[0])
        self.bn1 = nn.BatchNorm1d(392) # layer 1 output 
        
        #set2
        self.layer2 = nn.Linear(392,196)
        self.actv2 = nn.ReLU()
        self.do2 = nn.Dropout(dor[1])
        self.bn2 = nn.BatchNorm1d(196)

        #set 3

        self.layer3 = nn.Linear(196,98)
        self.actv3 = nn.ReLU()
        self.do3 = nn.Dropout(dor[2])
        self.bn3 = nn.BatchNorm1d(98)

        #set4
        self.layer4 = nn.Linear(98,49)
        self.actv4 = nn.ReLU()
        self.do4 = nn.Dropout(dor[0])
        self.bn4 = nn.BatchNorm1d(49)

        #set5
        self.layer5 = nn.Linear(49,24)
        self.actv5 = nn.ReLU()
        self.do5 = nn.Dropout(dor[3])
        self.bn5 = nn.BatchNorm1d(24)
        
        #ouput
        self.layer6 = nn.Linear(24,10)
        

    def forward(self, x):
        #Set1
        x = self.layer1(x)
        x = self.bn1(x)
        
        x = self.actv1(x)
        x = self.do1(x)

        #Set2
        x = self.layer2(x)
        x = self.bn2(x)
        
        x = self.actv2(x)
        x = self.do2(x)
        #Set3
        x = self.layer3(x)
        x = self.bn3(x)
        
        x = self.actv3(x)
        x = self.do3(x)
        #Set4
        x = self.layer4(x)
        x = self.bn4(x)
        
        x = self.actv4(x)
        x = self.do4(x)
        #Set5
        x = self.layer5(x)
        x = self.bn5(x)
        
        x = self.actv5(x)
        x = self.do5(x)
       
        #Set6
        x = self.layer6(x)
        return x
model = Model(input_dim).to(device=device)
print(model)


In [ ]:
for param in list(model.parameters()):
    print(param.shape)

# Input
#Bias


In [ ]:
modelPath = os.path.join(modelDir, subDir, f'torch_fifa_{torch.version.cuda}.pth')
modelPath

In [ ]:
#Loss function
#Compilation and # Fit()
loss_fn = nn.CrossEntropyLoss()
#Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = ALPHA)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, # learning rate decay
    mode = 'min',
    factor = LR_FACTOR,
    patience = LR_PATIENCE,
    min_lr = 1e-5
)


# always need to write 
# some lists to collect progress
loss = []
tloss = []
n_epoch = []
acc = []
tacc=[]
best_loss = np.inf
# loop for definedd epochs
for epoch in range(EPOCHS):
    model.train() # set model in training mode 
    epoch_loss = 0
    epoch_acc = 0
    tepoch_loss = 0
    tepoch_acc = 0
   
    for batch_idx, (train_X, train_y) in enumerate(train_loader):
        train_X = train_X.to(device)
        train_y = train_y.to(device)
        predict_proba = model(train_X) # make predict
        batch_loss = loss_fn(predict_proba, train_y) # calculate loss
        epoch_loss += (batch_loss - epoch_loss) / (batch_idx+1) # calculate running mean -> batch loss  for each batch -> then calculate avg loss for epoch
        #curr_loss = loss_fn(predict_proba, train_y)
        # Backpropagation
        optimizer.zero_grad() # Gradient set to Zero
        batch_loss.backward() # calculate loss in backpropagation
        optimizer.step() # move with the steps given by optimizer
        y_pred = predict_proba.argmax(dim=1).cpu().numpy()
        batch_acc = accuracy_score(train_y.cpu().numpy(), y_pred)
        epoch_acc += (batch_acc - epoch_acc)/(batch_idx+1) #accuracy for each batch -> then calculate avg accuracy for epoch
        
    loss.append(epoch_loss.data.item())
    acc.append(epoch_acc)

    model.eval()

    for batch_idx, (test_X, test_y) in enumerate(test_loader):
        test_X, test_y = test_X.to(device), test_y.to(device)
        test_proba = model(test_X)
        batch_loss = loss_fn(test_proba, test_y)
        tepoch_loss += (batch_loss - tepoch_loss)/ (batch_idx + 1)

        y_pred =test_proba.argmax(dim=1).cpu().numpy()
        batch_acc = accuracy_score(test_y.cpu().numpy(), y_pred)
        tepoch_acc += (batch_acc - tepoch_acc)/(batch_idx+1) #accuracy for each batch -> then calculate avg accuracy for epoch
    
    tacc.append(tepoch_acc)
    tloss.append(tepoch_loss.data.item())
    n_epoch.append(epoch)

    #LR Reduction Step
    scheduler.step(tepoch_loss)

    if tepoch_loss < best_loss:
        best_loss = tepoch_loss
        torch.save(model,modelPath)
    
    # loss.append(curr_loss.data.item())
    # y_pred = torch.argmax(predict_proba, dim=1).cpu().numpy()
    # curr_acc = accuracy_score(train_y.cpu().numpy(), y_pred)
    # acc.append(curr_acc)

    # model.eval()# set your model in eval mode
    # test_proba = model(test_X) # make prediction
    # test_loss = loss_fn(test_proba, test_y) # calculate loss
    # tloss.append(test_loss.data.item()) # append for plotting
    
    # y_pred = torch.argmax(test_proba, dim=1).cpu().numpy()
    # test_acc = accuracy_score(test_y.cpu().numpy(),y_pred)
    # tacc.append(test_acc)
    # n_epoch.append(epoch)

    if epoch % 10 == 0:
        print(f'Epoch:{epoch:>5d} | Loss: {epoch_loss:0.5f}/{tepoch_loss:0.5f}')
        print(f' Accuracy: {epoch_acc:0.5f}/{tepoch_acc:0.5f}')
        print(f'LR:{scheduler.get_last_lr()[0]:.5f}')
    

In [ ]:
loss_df = pd.DataFrame({ 
  'epoch':n_epoch ,
  'loss':loss,
  'test_loss':tloss,
  'acc':acc,
  'test_acc':tacc
})

loss_df.head()

In [ ]:
fn_plot_torch_hist(hist_df=loss_df) # drop 0.3 0.2 0.2 0.2 0.3

In [ ]:
del model

In [ ]:
model1 = torch.load(modelPath, weights_only = False)
model1.eval()
print(model)

In [ ]:
train_X = train_X.to(device)
train_y = train_y.to(device)
predict_proba = model1(train_X) 
y_pred = torch.argmax(predict_proba, dim=1).cpu().numpy()
curr_acc = accuracy_score(train_y.cpu().numpy(), y_pred)
print(curr_acc)


test_X = test_X.to(device)
test_y = test_y.to(device)
test_proba = model1(test_X) 
y_pred = torch.argmax(test_proba, dim=1).cpu().numpy()
test_acc = accuracy_score(test_y.cpu().numpy(),y_pred)
print(test_acc)